<a href="https://colab.research.google.com/github/robbybrodie/time_as_computation_cost/blob/main/notebooks/geodesics_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geodesics Experiment: Solar System Tests

This experiment tests our computational-capacity model against classic solar system observations: light bending, Shapiro delay, and Mercury perihelion precession.

## Theory

### Classical Solar System Tests
- **Light Bending**: Deflection of light rays passing near the Sun
- **Shapiro Delay**: Time delay of signals passing through gravitational fields  
- **Mercury Precession**: Advance of Mercury's perihelion per orbit

### PPN Predictions
- **Light deflection**: `δφ = (1+γ) * 4GM/(c²b)` where b is impact parameter
- **Shapiro delay**: `Δt = (1+γ) * 2GM/c³ * ln(r₁r₂/b²)` for signal path
- **Perihelion precession**: `Δφ = (2-β+2γ)/3 * 6πGM/(ac²(1-e²))` per orbit

### TACC Model Results
- **Our PPN values**: γ = κ/2, β = 1 (always)
- **GR limit**: κ = 2 → γ = 1, β = 1 → Einstein predictions
- **Observable scaling**: All effects scale predictably with κ

### Historical Context
These tests provided the first experimental confirmations of General Relativity and remain the most precise tests of gravity in the weak-field regime.

## Setup: Clone Repository and Install Dependencies

In [ ]:
# Clone and setup (idempotent)
import os, sys, subprocess, shutil, pathlib
REPO_URL = "https://github.com/robbybrodie/time_as_computation_cost.git"
REPO_NAME = "time_as_computation_cost"

if not pathlib.Path(REPO_NAME).exists():
    !git clone $REPO_URL

%cd $REPO_NAME

# Install package
if (pathlib.Path("pyproject.toml")).exists():
    !pip install -e .

# Set random seed for reproducibility
import numpy as np, random
np.random.seed(42)
random.seed(42)

## Run Geodesics Experiment

In [ ]:
from experiments.run_geodesics import main
main()

## Display Results

In [ ]:
from IPython.display import Image, display
from pathlib import Path

# Display generated plots
output_dir = Path("experiments/out/geodesics")

print("Light Bending vs Impact Parameter:")
display(Image(str(output_dir / "light_bending.png")))

print("\nShapiro Delay vs Impact Parameter:")
display(Image(str(output_dir / "shapiro_delay.png")))

print("\nMercury Perihelion Precession:")
display(Image(str(output_dir / "mercury_precession.png")))

print("\nSolar System Tests Summary:")
display(Image(str(output_dir / "solar_system_tests.png")))

# Display numerical results
print("\nNumerical Results:")
with open(output_dir / "results.txt", 'r') as f:
    print(f.read())

## Optional: Custom Solar System Parameters

In [ ]:
# Explore different planetary systems
import numpy as np
import matplotlib.pyplot as plt
from tacc import geodesics

# Solar system parameters
GM_sun = 1.327e20  # m^3/s^2
c = 2.998e8  # m/s

# Test with different planetary parameters
planets = {
    'Mercury': {'a': 0.387, 'e': 0.206},
    'Venus': {'a': 0.723, 'e': 0.007},
    'Earth': {'a': 1.000, 'e': 0.017},
    'Mars': {'a': 1.524, 'e': 0.094}
}

gamma_values = [0.5, 1.0, 1.5, 2.0]
beta = 1.0  # Always 1 in our model

plt.figure(figsize=(12, 8))

for i, (planet_name, params) in enumerate(planets.items()):
    plt.subplot(2, 2, i+1)
    
    precessions = []
    for gamma in gamma_values:
        precession = geodesics.perihelion_precession(params['a'], params['e'], beta, gamma, GM_sun, c)
        # Convert to arcseconds per century
        if planet_name == 'Mercury':
            period_days = 88
        elif planet_name == 'Venus':
            period_days = 225
        elif planet_name == 'Earth':
            period_days = 365
        elif planet_name == 'Mars':
            period_days = 687
        
        precession_arcsec = precession * 206265 * 100 / (period_days * 365.25 * 24 * 3600) * (100 * 365.25 * 24 * 3600)
        precessions.append(precession_arcsec)
    
    plt.plot(gamma_values, precessions, 'o-', linewidth=2, markersize=6)
    plt.xlabel('γ (PPN Parameter)')
    plt.ylabel('Precession (arcsec/century)')
    plt.title(f'{planet_name} (a={params["a"]} AU, e={params["e"]})')
    plt.grid(True, alpha=0.3)
    
    # Add observed value for Mercury
    if planet_name == 'Mercury':
        plt.axhline(y=43.1, color='r', linestyle='--', alpha=0.7, label='Observed')
        plt.legend()

plt.suptitle('Perihelion Precession for Different Planets')
plt.tight_layout()
plt.show()

print("Key Observations:")
print("- Mercury shows the largest precession effect due to its proximity to the Sun")
print("- The effect scales with orbital parameters and proximity to the gravitational source")
print("- γ=1 (Einstein's GR) best matches the observed Mercury precession anomaly")

## Custom Light Bending Analysis

In [ ]:
# Detailed light bending analysis
import numpy as np
import matplotlib.pyplot as plt
from tacc import geodesics

# Solar parameters
GM_sun = 1.327e20  # m^3/s^2
R_sun = 6.96e8  # solar radius in meters
c = 2.998e8  # m/s

# Famous eclipse measurements
eclipse_data = {
    '1919 Sobral': {'measured': 1.98, 'error': 0.12},  # arcseconds
    '1919 Principe': {'measured': 1.61, 'error': 0.30},
    'Modern average': {'measured': 1.750, 'error': 0.009}
}

# Calculate predictions for different gamma values at solar limb
gamma_values = np.linspace(0.5, 2.0, 100)
predictions = []

for gamma in gamma_values:
    # At solar limb: b ≈ R_sun, so b/rs ≈ 1
    deflection = geodesics.null_geodesic_deflection(1.0, gamma, GM_sun, c)
    deflection_arcsec = deflection * 206265
    predictions.append(deflection_arcsec)

plt.figure(figsize=(10, 6))
plt.plot(gamma_values, predictions, 'b-', linewidth=2, label='Our Model')

# Plot historical measurements
colors = ['red', 'green', 'purple']
for i, (name, data) in enumerate(eclipse_data.items()):
    plt.axhline(y=data['measured'], color=colors[i], linestyle='--', 
               label=f'{name}: {data["measured"]}±{data["error"]}"')
    plt.fill_between(gamma_values, 
                    data['measured'] - data['error'],
                    data['measured'] + data['error'],
                    color=colors[i], alpha=0.1)

# Mark Einstein's prediction (gamma=1)
einstein_pred = geodesics.null_geodesic_deflection(1.0, 1.0, GM_sun, c) * 206265
plt.axvline(x=1.0, color='black', linestyle=':', alpha=0.7, label='Einstein (γ=1)')
plt.scatter([1.0], [einstein_pred], color='black', s=100, zorder=10)

plt.xlabel('γ (PPN Parameter)')
plt.ylabel('Light Deflection at Solar Limb (arcseconds)')
plt.title('Light Bending: Model vs Historical Measurements')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim([1.0, 2.5])

plt.tight_layout()
plt.show()

print(f"Einstein's prediction (γ=1): {einstein_pred:.3f} arcseconds")
print("Historical context:")
print("- 1919 eclipse expeditions confirmed Einstein's prediction")
print("- Modern measurements agree with GR to high precision")
print("- Our model reduces to GR when γ=1 (κ=2)")

## Interactive Setup

In [ ]:
# Setup for comprehensive interactive analysis
import numpy as np, matplotlib.pyplot as plt
from pathlib import Path
import sys

# Ensure we can import modules
repo_root = Path().resolve()
sys.path.insert(0, str(repo_root / "src"))

from tacc import geodesics
from experiments.run_geodesics import main

print("Interactive geodesics analysis modules loaded successfully!")

## Comprehensive Solar System Analysis

In [ ]:
def comprehensive_solar_system_analysis():
    """Comprehensive analysis of all solar system tests"""
    
    # Physical constants
    GM_sun = 1.327e20  # m^3/s^2
    R_sun = 6.96e8     # solar radius in meters  
    c = 2.998e8        # m/s
    
    # Parameter ranges
    gamma_range = np.linspace(0.5, 1.5, 100)
    beta = 1.0  # Always 1 in our model
    
    # Create comprehensive visualization
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # Light bending analysis
    impact_params = np.logspace(0, 2, 50)  # In units of solar radii
    colors = ['red', 'blue', 'green', 'orange']
    gamma_values = [0.5, 1.0, 1.2, 1.5]
    
    for i, gamma in enumerate(gamma_values):
        deflections = []
        for b_ratio in impact_params:
            deflection = geodesics.null_geodesic_deflection(b_ratio, gamma, GM_sun, c)
            deflections.append(deflection * 206265)  # Convert to arcseconds
        ax1.loglog(impact_params, deflections, color=colors[i], linewidth=2, 
                  label=f'γ={gamma} (κ={2*gamma})')
    
    # Add solar limb measurement
    einstein_deflection = geodesics.null_geodesic_deflection(1.0, 1.0, GM_sun, c) * 206265
    ax1.scatter([1.0], [einstein_deflection], color='black', s=100, zorder=10, marker='*')
    ax1.axvline(x=1.0, color='k', linestyle='--', alpha=0.7, label='Solar limb')
    ax1.set_xlabel('Impact Parameter (Solar Radii)')
    ax1.set_ylabel('Light Deflection (arcseconds)')
    ax1.set_title('Light Bending vs Impact Parameter')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Shapiro delay analysis
    for i, gamma in enumerate(gamma_values):
        delays = []
        for b_ratio in impact_params:
            # Approximate Shapiro delay for superior conjunction
            r1 = r2 = 1.5e11  # Earth-Sun distance (approximate)
            b = b_ratio * R_sun
            delay = geodesics.shapiro_delay(r1, r2, b, gamma, GM_sun, c)
            delays.append(delay * 1e6)  # Convert to microseconds
        ax2.loglog(impact_params, delays, color=colors[i], linewidth=2, 
                  label=f'γ={gamma}')
    
    ax2.set_xlabel('Impact Parameter (Solar Radii)')
    ax2.set_ylabel('Shapiro Delay (μs)')
    ax2.set_title('Shapiro Time Delay')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Mercury precession analysis
    mercury_a = 0.387  # AU
    mercury_e = 0.206
    precessions = []
    
    for gamma in gamma_range:
        precession_per_orbit = geodesics.perihelion_precession(mercury_a, mercury_e, beta, gamma, GM_sun, c)
        # Convert to arcseconds per century
        mercury_period_days = 88
        orbits_per_century = 100 * 365.25 / mercury_period_days
        precession_arcsec_century = precession_per_orbit * 206265 * orbits_per_century
        precessions.append(precession_arcsec_century)
    
    ax3.plot(gamma_range, precessions, 'purple', linewidth=2, label='Our model')
    ax3.axhline(y=43.1, color='r', linestyle='--', alpha=0.7, label='Observed (43.1"/century)')
    ax3.axvline(x=1.0, color='k', linestyle=':', alpha=0.7, label='Einstein (γ=1)')
    ax3.fill_between([0.9999, 1.0001], 42, 44, alpha=0.2, color='green', label='Observational uncertainty')
    ax3.set_xlabel('γ (PPN Parameter)')
    ax3.set_ylabel('Mercury Precession (arcsec/century)')
    ax3.set_title('Mercury Perihelion Precession')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    ax3.set_ylim([30, 60])
    
    # Combined constraint analysis
    # Calculate constraint strengths for each test
    light_bending_1919 = 1.75  # arcseconds, approximate average
    light_bending_error = 0.1
    mercury_observed = 43.1
    mercury_error = 0.5  # approximate
    
    # Light bending constraint on gamma
    predicted_bending = []
    for gamma in gamma_range:
        bending = geodesics.null_geodesic_deflection(1.0, gamma, GM_sun, c) * 206265
        predicted_bending.append(bending)
    
    light_constraint = np.exp(-0.5 * ((np.array(predicted_bending) - light_bending_1919) / light_bending_error)**2)
    mercury_constraint = np.exp(-0.5 * ((np.array(precessions) - mercury_observed) / mercury_error)**2)
    
    # Combined constraint
    combined_constraint = light_constraint * mercury_constraint
    combined_constraint /= np.max(combined_constraint)
    
    ax4.plot(gamma_range, light_constraint, 'r-', linewidth=2, label='Light bending')
    ax4.plot(gamma_range, mercury_constraint, 'b-', linewidth=2, label='Mercury precession')
    ax4.plot(gamma_range, combined_constraint, 'k-', linewidth=3, label='Combined')
    ax4.axvline(x=1.0, color='g', linestyle='--', alpha=0.7, label='Einstein (γ=1)')
    ax4.fill_between(gamma_range, 0, combined_constraint, alpha=0.3, color='gray')
    ax4.set_xlabel('γ (PPN Parameter)')
    ax4.set_ylabel('Constraint Strength')
    ax4.set_title('Combined Solar System Constraints')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Find best-fit gamma
    best_fit_idx = np.argmax(combined_constraint)
    best_fit_gamma = gamma_range[best_fit_idx]
    best_fit_kappa = 2 * best_fit_gamma
    
    print("Solar System Test Results:")
    print(f"  Best-fit γ = {best_fit_gamma:.6f}")
    print(f"  Corresponding κ = {best_fit_kappa:.6f}")
    print(f"  Light deflection prediction: {predicted_bending[best_fit_idx]:.3f} arcsec")
    print(f"  Mercury precession prediction: {precessions[best_fit_idx]:.1f} arcsec/century")
    print(f"  Agreement with Einstein GR: {abs(best_fit_gamma - 1.0)*100:.4f}%")

print("Running comprehensive solar system analysis:")
comprehensive_solar_system_analysis()

## Historical Context & Measurements

In [ ]:
def historical_measurements_analysis():
    """Analysis of historical measurements and their significance"""
    
    # Historical data
    historical_data = {
        'Light Bending': {
            '1919 Sobral': {'value': 1.98, 'error': 0.12, 'year': 1919},
            '1919 Principe': {'value': 1.61, 'error': 0.30, 'year': 1919},
            '1922 Australia': {'value': 1.77, 'error': 0.40, 'year': 1922},
            '1973 Mauritania': {'value': 1.66, 'error': 0.18, 'year': 1973},
            'Modern VLBI': {'value': 1.750, 'error': 0.009, 'year': 2000}
        },
        'Mercury Precession': {
            'Le Verrier (1859)': {'value': 43.0, 'error': 1.0, 'year': 1859},
            'Newcomb (1895)': {'value': 43.0, 'error': 0.5, 'year': 1895}, 
            'Modern': {'value': 43.11, 'error': 0.21, 'year': 1990}
        }
    }
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    # Light bending historical evolution
    years = [data['year'] for data in historical_data['Light Bending'].values()]
    values = [data['value'] for data in historical_data['Light Bending'].values()]
    errors = [data['error'] for data in historical_data['Light Bending'].values()]
    names = list(historical_data['Light Bending'].keys())
    
    ax1.errorbar(years, values, yerr=errors, fmt='o-', linewidth=2, markersize=8, capsize=5)
    ax1.axhline(y=1.75, color='r', linestyle='--', alpha=0.7, label='Einstein prediction')
    ax1.axhline(y=0.875, color='b', linestyle=':', alpha=0.7, label='Newtonian (half-deflection)')
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Light Deflection (arcseconds)')
    ax1.set_title('Historical Light Bending Measurements')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim([0, 3])
    
    # Add measurement labels
    for i, (year, value, name) in enumerate(zip(years, values, names)):
        ax1.annotate(name.split()[1] if len(name.split()) > 1 else name, 
                    (year, value), xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    # Mercury precession historical evolution
    merc_years = [data['year'] for data in historical_data['Mercury Precession'].values()]
    merc_values = [data['value'] for data in historical_data['Mercury Precession'].values()]
    merc_errors = [data['error'] for data in historical_data['Mercury Precession'].values()]
    merc_names = list(historical_data['Mercury Precession'].keys())
    
    ax2.errorbar(merc_years, merc_values, yerr=merc_errors, fmt='s-', linewidth=2, 
                markersize=8, capsize=5, color='purple')
    ax2.axhline(y=43.11, color='r', linestyle='--', alpha=0.7, label='Einstein prediction')
    ax2.set_xlabel('Year')
    ax2.set_ylabel('Mercury Precession (arcsec/century)')
    ax2.set_title('Historical Mercury Precession Measurements')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim([40, 46])
    
    # Measurement precision evolution
    all_years = years + merc_years
    light_precision = [1/err for err in errors] + [0]*len(merc_errors)
    mercury_precision = [0]*len(errors) + [1/err for err in merc_errors]
    
    ax3.semilogy(years, [1/err for err in errors], 'bo-', linewidth=2, markersize=8, label='Light bending')
    ax3.semilogy(merc_years, [1/err for err in merc_errors], 'rs-', linewidth=2, markersize=8, label='Mercury precession')
    ax3.set_xlabel('Year')
    ax3.set_ylabel('Measurement Precision (1/σ)')
    ax3.set_title('Measurement Precision Evolution')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Constraint on gamma over time
    gamma_range = np.linspace(0.5, 1.5, 200)
    
    # 1919 constraint (combined Sobral + Principe)
    avg_1919 = np.average([1.98, 1.61], weights=[1/0.12**2, 1/0.30**2])
    err_1919 = 1/np.sqrt(1/0.12**2 + 1/0.30**2)
    
    GM_sun = 1.327e20
    c = 2.998e8
    
    constraint_1919 = []
    constraint_modern = []
    
    for gamma in gamma_range:
        pred_deflection = geodesics.null_geodesic_deflection(1.0, gamma, GM_sun, c) * 206265
        
        # 1919 constraint
        likelihood_1919 = np.exp(-0.5 * ((pred_deflection - avg_1919) / err_1919)**2)
        constraint_1919.append(likelihood_1919)
        
        # Modern constraint
        likelihood_modern = np.exp(-0.5 * ((pred_deflection - 1.750) / 0.009)**2)
        constraint_modern.append(likelihood_modern)
    
    ax4.plot(gamma_range, constraint_1919, 'b-', linewidth=2, label='1919 eclipse')
    ax4.plot(gamma_range, constraint_modern, 'r-', linewidth=2, label='Modern VLBI')
    ax4.axvline(x=1.0, color='k', linestyle='--', alpha=0.7, label='Einstein (γ=1)')
    ax4.fill_between(gamma_range, 0, constraint_1919, alpha=0.3, color='blue')
    ax4.set_xlabel('γ (PPN Parameter)')
    ax4.set_ylabel('Constraint Likelihood')
    ax4.set_title('Constraint Evolution on γ')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("Historical Context:")
    print("  • 1919: First confirmation of GR - dramatic shift from Newtonian physics")
    print("  • Mercury precession: Known anomaly since 1859, explained by Einstein in 1915")
    print("  • Modern era: VLBI and spacecraft tracking give unprecedented precision")
    print("  • Current precision constrains γ to ~10⁻⁵ level")
    
    print(f"\n1919 Eclipse Results:")
    print(f"  Combined average: {avg_1919:.2f} ± {err_1919:.2f} arcsec")
    print(f"  Einstein prediction: 1.75 arcsec")
    print(f"  Agreement: {abs(avg_1919-1.75)/err_1919:.1f}σ")

print("Running historical measurements analysis:")
historical_measurements_analysis()

## Key Insights

1. **Solar System Validation**: All three classic tests (light bending, Shapiro delay, Mercury precession) provide consistent constraints on γ

2. **Historical Significance**: 1919 eclipse measurements provided the first experimental confirmation of General Relativity

3. **Modern Precision**: Current measurements constrain γ = 1.000000 ± 0.000023, requiring κ ≈ 2 with extraordinary precision

4. **Scaling Relationships**: All effects scale predictably with γ, providing multiple independent tests

5. **Parameter Degeneracy**: β = 1 (always) in our model means Mercury precession depends only on γ

## Physical Interpretation

- **Light Bending**: Spacetime curvature deflects photon trajectories - scales as (1+γ)/2
- **Shapiro Delay**: Time dilation in gravitational fields delays signal propagation
- **Mercury Precession**: Orbital dynamics in curved spacetime - most sensitive to strong-field effects
- **κ Parameter**: Controls deviation from GR - observationally constrained to κ ≈ 2

**Note**: These are tests of the conceptual framework using well-established physics results!

## Troubleshooting

**Common Issues:**
- Large numbers in calculations - results are in SI units
- Import errors indicate repository setup problems
- If plots don't display properly, ensure matplotlib is installed

**Expected Output:**
- Comprehensive 4-panel solar system analysis with parameter sweeps
- Historical measurement evolution showing precision improvements
- Combined constraint analysis showing γ ≈ 1 requirement
- Quantitative comparison with Einstein's predictions

**Physical Insights:**
- Light bending scales as (1+γ) - our model allows tuning via γ
- Shapiro delay also depends on (1+γ) 
- Mercury precession scales as (2-β+2γ), with β=1 in our model
- All effects reduce to Einstein's GR when γ=1 (κ=2)

**Key Results:**
- Historical measurements validated Einstein's predictions with increasing precision
- Modern constraints require κ ≈ 2.000000 for consistency with observations
- The computational capacity parameter provides a framework for testing alternative gravity theories